In [1]:
# Imports
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import requests
import os

In [ ]:
df_1 = pd.read_csv('Data/Oct_Dec_2022_Lethbridge_Police_Service_report.csv')
df_2 = pd.read_csv('Data/Jan_Mar_2023_Lethbridge_Police_Service_report.csv')
df_3 = pd.read_csv('Data/Apr_Jun_2023_Lethbridge_Police_Service_report.csv')

df = pd.concat([df_1, df_2, df_3])
df = df.drop_duplicates()
df.head()

In [ ]:
df = df.dropna(subset=['blocksizedAddress'], axis=0)
df = df[~df['parentIncidentType'].isin(['Proactive Policing', 'Traffic'])]
df = df.drop(['updateDate','incidentType', 'postalCode'], axis=1)
df = df[df['blocksizedAddress'].str.contains('Block')]

In [ ]:
df['address'] = df['blocksizedAddress'].str.replace('Block ', '') + ', ' + df['city'] + ', ' + df['state'] + ', Canada'

In [ ]:
# Creating pivot table of all unique addresses
df_address = df.pivot_table(index='address', values='date', aggfunc='count').sort_values('date', ascending=False).reset_index()
df_address.columns = ['address', 'incidentCount']

In [ ]:
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv('GOOGLE_API_KEY2')

In [ ]:
lat = []
lng = []

# For loop to pull coordinates for every address
for i in df_address['address']:
    
    outputFormat = 'json'
    parameters = {
        'key': api_key,
        'address': i
    }
    
    endpoint = f'https://maps.googleapis.com/maps/api/geocode/{outputFormat}?'
    response = requests.get(endpoint, params = parameters).json()

    lat.append(response['results'][0]['geometry']['location']['lat'])
    lng.append(response['results'][0]['geometry']['location']['lng'])

df_address.insert(1,'lat',lat)
df_address.insert(2,'lng',lng)
df_address.head()

In [2]:
df_address.to_csv('df_address.csv', index = False)

NameError: name 'df_address' is not defined